En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "../farmers-protest-tweets-2021-2-4.json"

In [14]:
import re
import json
import emoji
import random
import orjson
import unicodedata
import polars as pl
import pandas as pd
from collections import Counter
from typing import List, Iterable, Callable
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor


# Local libraries
from utils import twitter_schema
from benchmark import profile_performance, profile_detailed

# Challenge

El archivo [SOLUTION.md](SOLUTION.md) contiene la explicación detallada del paso apaso de la solución

## Ambiente de desarrollo

Se utiliza make para desarrollar el proyecto. Puedes ver las opciones disponibles en el archivo [Makefile](Makefile).

## Data Analisis

Se realiza un analisis de los datos para comprender su estructura y posibles problemas de calidad. ver detalle en la sección 2 del archivo [SOLUTION.md](../SOLUTION.md) 


In [ ]:
def analyze_twitter_nuances(file_path, sample_size=20000):
    print(f"--- 🕵️‍♂️ Deep Dive Analysis: {file_path} ---")

    data = []
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i >= sample_size:
                    break
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
    except FileNotFoundError:
        print("Error: Archivo no encontrado.")
        return

    df = pd.DataFrame(data)

    # ==========================================
    # 1. ANÁLISIS DE TRUNCAMIENTO (Extended Mode)
    # ==========================================
    print("\n[1] ANÁLISIS DE TRUNCAMIENTO")

    # Verificar si existen claves nativas de la API v1.1
    has_truncated_key = "truncated" in df.columns
    has_extended_tweet = "extended_tweet" in df.columns

    if has_truncated_key:
        truncated_count = df["truncated"].sum() if df["truncated"].dtype == bool else 0
        print(
            f" - Tweets marcados como 'truncated': {truncated_count} ({truncated_count / len(df):.2%})"
        )

        if has_extended_tweet:
            extended_count = df["extended_tweet"].notnull().sum()
            print(f" - Tweets con objeto 'extended_tweet' disponible: {extended_count}")
    else:
        print(
            " - La clave 'truncated' NO existe en este dataset (probablemente ya fue procesado/aplanado)."
        )

    # Verificar visualmente si el contenido parece cortado
    # Los tweets truncados suelen terminar en "..." o un enlace t.co
    df["ends_with_ellipsis"] = df["content"].astype(str).str.strip().str.endswith("…")
    suspicious_truncation = df["ends_with_ellipsis"].sum()
    print(f" - Tweets que terminan visualmente en '…': {suspicious_truncation}")

    # ==========================================
    # 2. ANÁLISIS DE RETWEETS (Duplicidad)
    # ==========================================
    print("\n[2] ANÁLISIS DE RETWEETS")

    # Detectar RTs
    # Opción A: Clave 'retweeted_status' (Standard API)
    if "retweeted_status" in df.columns:
        rts_count = df["retweeted_status"].notnull().sum()
        print(f" - Detectados por metadato 'retweeted_status': {rts_count}")

    # Opción B: Texto empieza con "RT @"
    df["is_rt_text"] = df["content"].astype(str).str.startswith("RT @")
    rts_text_count = df["is_rt_text"].sum()
    print(
        f" - Detectados por texto ('RT @...'): {rts_text_count} ({rts_text_count / len(df):.2%})"
    )

    if rts_text_count > 0:
        print(
            "   -> CONCLUSIÓN: Los Retweets están presentes. Q2 y Q3 estarán inflados por repetición."
        )

    # ==========================================
    # 3. MENCIONES: TEXTO VS METADATA (Q3)
    # ==========================================
    print("\n[3] COMPARATIVA DE MENCIONES (Q3)")

    # Función para extraer menciones con Regex (enfoque ingenuo)
    def extract_regex_mentions(text):
        return set(re.findall(r"@(\w+)", str(text)))

    # Función para extraer menciones de Metadata (enfoque robusto)
    def extract_meta_mentions(mentions_list):
        if not isinstance(mentions_list, list):
            return set()
        return set(
            m.get("username")
            for m in mentions_list
            if isinstance(m, dict) and m.get("username")
        )

    # Aplicar comparativa en una muestra pequeña para velocidad
    sample_df = df.head(1000).copy()

    sample_df["regex_mentions"] = sample_df["content"].apply(extract_regex_mentions)
    sample_df["meta_mentions"] = sample_df["mentionedUsers"].apply(
        extract_meta_mentions
    )

    # Buscar discrepancias
    # Casos donde Metadata tiene ALGO pero Regex NO (Menciones invisibles/Reply)
    sample_df["hidden_mentions"] = sample_df.apply(
        lambda x: x["meta_mentions"] - x["regex_mentions"], axis=1
    )
    hidden_count = sample_df[sample_df["hidden_mentions"].astype(bool)].shape[0]

    # Casos donde Regex tiene ALGO pero Metadata NO (Falsos positivos, emails, usuarios suspendidos)
    sample_df["fake_mentions"] = sample_df.apply(
        lambda x: x["regex_mentions"] - x["meta_mentions"], axis=1
    )
    fake_count = sample_df[sample_df["fake_mentions"].astype(bool)].shape[0]

    print(f"Análisis sobre {len(sample_df)} registros:")
    print(
        f" - Casos donde Metadata detecta usuarios que Regex NO ve (Hidden/Reply): {hidden_count}"
    )
    if hidden_count > 0:
        example = sample_df[sample_df["hidden_mentions"].astype(bool)].iloc[0]
        print(
            f"   Ejemplo Hidden -> Texto: '{example['content'][:50]}...' | Meta: {example['meta_mentions']}"
        )

    print(
        f" - Casos donde Regex detecta '@' que NO son usuarios válidos en Metadata: {fake_count}"
    )
    if fake_count > 0:
        example = sample_df[sample_df["fake_mentions"].astype(bool)].iloc[0]
        print(
            f"   Ejemplo Falso Positivo -> Texto: '{example['content'][:50]}...' | Regex: {example['regex_mentions']}"
        )


def advanced_analysis(file_path):
    print(f"--- 🔬 Análisis Forense Avanzado: {file_path} ---")

    usernames_raw = []
    usernames_normalized = []

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    tweet = json.loads(line)
                    u = tweet.get("user", {}).get("username")
                    if u:
                        usernames_raw.append(u)
                        # Normalización NFKC + Lowercase
                        usernames_normalized.append(
                            unicodedata.normalize("NFKC", u).lower()
                        )
                except Exception:
                    continue
    except FileNotFoundError:
        print("Archivo no encontrado")
        return

    # 1. CHECK DE CASE SENSITIVITY Y UNICODE
    unique_raw = len(set(usernames_raw))
    unique_norm = len(set(usernames_normalized))

    print("\n[1] INTEGRIDAD DE ENTIDADES (Usernames)")
    print(f" - Usuarios únicos (Crudo): {unique_raw}")
    print(f" - Usuarios únicos (Normalizado + Lower): {unique_norm}")
    diff = unique_raw - unique_norm
    if diff > 0:
        print(
            f" ⚠️ ALERTA: Se detectaron {diff} duplicados por falta de normalización/mayúsculas."
        )
        print(" -> ACCIÓN: Es MANDATORIO aplicar .lower() y unicodedata.")
    else:
        print(
            " -> OK: No se detectaron colisiones, pero es buena práctica implementarlo."
        )

    # 2. DETECCIÓN DE BOTS (OUTLIERS Q1)
    print("\n[2] DISTRIBUCIÓN DE ACTIVIDAD (Q1)")
    counts = Counter(usernames_raw)
    top_5 = counts.most_common(5)

    df = pd.Series(list(counts.values()))
    p99 = df.quantile(0.99)
    max_tweets = df.max()

    print(f" - Top 5 Usuarios más activos:\n   {top_5}")
    print(f" - El 99% de usuarios tiene menos de {p99:.0f} tweets.")
    print(f" - El usuario #1 tiene {max_tweets} tweets.")

    if max_tweets > (p99 * 10):
        print(
            f" ⚠️ ALERTA: El usuario top tiene una actividad {max_tweets / p99:.1f}x mayor al promedio."
        )
        print(
            " -> OBSERVACIÓN: Probable Bot. Documentar en SOLUTION.md que esto sesga los resultados."
        )

    # 3. EMPATES (TIE-BREAKING)
    print("\n[3] RIESGO DE EMPATES EN EL CORTE")
    # Ver si hay muchos usuarios con el mismo conteo en el borde del top 10
    counts_values = list(counts.values())
    counts_freq = Counter(counts_values)

    # Imaginemos que el corte del top 10 es alrededor de X tweets
    sorted_counts = sorted(counts_values, reverse=True)
    if len(sorted_counts) > 10:
        val_at_10 = sorted_counts[9]  # El valor del décimo lugar
        users_at_cutoff = counts_freq[val_at_10]
        print(f" - Valor de corte (Puesto #10): {val_at_10} tweets")
        print(f" - Cuántos usuarios tienen exactamente ese valor: {users_at_cutoff}")

        if users_at_cutoff > 1:
            print(" ⚠️ ALERTA CRÍTICA: Hay EMPATE en el puesto #10.")
            print(
                " -> ACCIÓN: Tu código DEBE tener un criterio de desempate (ej: alfabético) o los tests fallarán aleatoriamente."
            )


def discover_anomalies(file_path, sample_size=15000):
    print(f"--- Iniciando Análisis Profundo: {file_path} (n={sample_size}) ---")

    data = []
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i < sample_size:
                    data.append(line)
                else:
                    r = random.randint(0, i)
                    if r < sample_size:
                        data[r] = line
    except FileNotFoundError:
        print(f"Error: El archivo {file_path} no existe.")
        return

    parsed = []
    corrupt_lines = 0
    for line in data:
        try:
            parsed.append(json.loads(line))
        except Exception:
            corrupt_lines += 1

    if corrupt_lines:
        print(f"[ALERTA] Se detectaron {corrupt_lines} líneas corruptas en la muestra.")

    # Normalización para análisis
    df = pd.json_normalize(parsed)

    print("\n1. INTEGRIDAD DE COLUMNAS CLAVE")
    columns_to_check = ["date", "content", "mentionedUsers", "user.username", "user.id"]
    for col in columns_to_check:
        if col in df.columns:
            types = df[col].apply(lambda x: type(x).__name__).value_counts()
            nulls = df[col].isnull().sum()
            print(f"- '{col}': {len(types)} tipos detectados. Nulos: {nulls}")
        else:
            print(f"- [ERROR] Columna '{col}' NO encontrada.")

    print("\n2. ANÁLISIS DE USUARIOS (Q1 & Q3)")
    # Verificar si un username tiene múltiples IDs (cambio de handle)
    user_consistency = df.groupby("user.username")["user.id"].nunique()
    inconsistent = user_consistency[user_consistency > 1]
    print(f"- Usuarios con más de un ID: {len(inconsistent)}")

    # Calcular potencial de String Interning
    total_names = len(df["user.username"])
    unique_names = df["user.username"].nunique()
    print(
        f"- Ratio de Repetición de Usernames: {total_names / unique_names:.2f}x (Alto ratio justifica sys.intern)"
    )

    print("\n3. ANÁLISIS DE MENCIONES (Q3)")
    # Validar estructura interna de mentionedUsers
    mentions_data = df["mentionedUsers"].dropna()
    has_nested_nulls = mentions_data.apply(
        lambda x: any(m.get("username") is None for m in x if isinstance(m, dict))
    ).sum()
    print(f"- Registros con listas de menciones válidas: {len(mentions_data)}")
    print(f"- Listas con objetos internos nulos: {has_nested_nulls}")

    print("\n4. ANÁLISIS DE EMOJIS COMPLEJOS (Q2)")
    all_emojis = []
    complex_count = 0
    for txt in df["content"].dropna():
        # emoji_list devuelve información detallada de cada emoji
        found = emoji.emoji_list(txt)
        for e in found:
            char = e["emoji"]
            all_emojis.append(char)
            # Si tiene más de un componente unicode o caracteres especiales de unión
            if len(char) > 1 or "\u200d" in char:
                complex_count += 1

    print(f"- Total emojis detectados: {len(all_emojis)}")
    print(f"- Emojis complejos (ZWJ/Multi-char): {complex_count}")
    if all_emojis:
        print(f"- Top 3 Emojis en muestra: {Counter(all_emojis).most_common(3)}")

    print("\n5. VALORES EXTREMOS Y FECHAS")
    df["date_parsed"] = pd.to_datetime(df["date"], errors="coerce")
    print(f"- Rango temporal: {df['date_parsed'].min()} a {df['date_parsed'].max()}")
    print(f"- Tweets por fuera de 2021: {len(df[df['date_parsed'].dt.year != 2021])}")

In [4]:
analyze_twitter_nuances(file_path)

--- 🕵️‍♂️ Deep Dive Analysis: ../farmers-protest-tweets-2021-2-4.json ---

[1] ANÁLISIS DE TRUNCAMIENTO
 - La clave 'truncated' NO existe en este dataset (probablemente ya fue procesado/aplanado).
 - Tweets que terminan visualmente en '…': 4

[2] ANÁLISIS DE RETWEETS
 - Detectados por texto ('RT @...'): 10 (0.05%)
   -> CONCLUSIÓN: Los Retweets están presentes. Q2 y Q3 estarán inflados por repetición.

[3] COMPARATIVA DE MENCIONES (Q3)
Análisis sobre 1000 registros:
 - Casos donde Metadata detecta usuarios que Regex NO ve (Hidden/Reply): 14
   Ejemplo Hidden -> Texto: '.@RakeshTikaitBKU बोले- संसद जाकर ट्रैक्टर चलाएंगे...' | Meta: {'RakeshTikaitBKU', 'KumarKunalmedia'}
 - Casos donde Regex detecta '@' que NO son usuarios válidos en Metadata: 15
   Ejemplo Falso Positivo -> Texto: '.@RakeshTikaitBKU बोले- संसद जाकर ट्रैक्टर चलाएंगे...' | Regex: {'RakeshTikaitBKU', 'kumarkunalmedia'}


In [5]:
advanced_analysis(file_path)

--- 🔬 Análisis Forense Avanzado: ../farmers-protest-tweets-2021-2-4.json ---

[1] INTEGRIDAD DE ENTIDADES (Usernames)
 - Usuarios únicos (Crudo): 26519
 - Usuarios únicos (Normalizado + Lower): 26519
 -> OK: No se detectaron colisiones, pero es buena práctica implementarlo.

[2] DISTRIBUCIÓN DE ACTIVIDAD (Q1)
 - Top 5 Usuarios más activos:
   [('jot__b', 1019), ('rebelpacifist', 850), ('MaanDee08215437', 830), ('Gurpreetd86', 636), ('GurmVicky', 597)]
 - El 99% de usuarios tiene menos de 56 tweets.
 - El usuario #1 tiene 1019 tweets.
 ⚠️ ALERTA: El usuario top tiene una actividad 18.2x mayor al promedio.
 -> OBSERVACIÓN: Probable Bot. Documentar en SOLUTION.md que esto sesga los resultados.

[3] RIESGO DE EMPATES EN EL CORTE
 - Valor de corte (Puesto #10): 490 tweets
 - Cuántos usuarios tienen exactamente ese valor: 1


In [6]:
discover_anomalies(file_path)

--- Iniciando Análisis Profundo: ../farmers-protest-tweets-2021-2-4.json (n=15000) ---

1. INTEGRIDAD DE COLUMNAS CLAVE
- 'date': 1 tipos detectados. Nulos: 0
- 'content': 1 tipos detectados. Nulos: 0
- 'mentionedUsers': 2 tipos detectados. Nulos: 10107
- 'user.username': 1 tipos detectados. Nulos: 0
- 'user.id': 1 tipos detectados. Nulos: 0

2. ANÁLISIS DE USUARIOS (Q1 & Q3)
- Usuarios con más de un ID: 0
- Ratio de Repetición de Usernames: 2.18x (Alto ratio justifica sys.intern)

3. ANÁLISIS DE MENCIONES (Q3)
- Registros con listas de menciones válidas: 4893
- Listas con objetos internos nulos: 0

4. ANÁLISIS DE EMOJIS COMPLEJOS (Q2)
- Total emojis detectados: 5685
- Emojis complejos (ZWJ/Multi-char): 1320
- Top 3 Emojis en muestra: [('🙏', 663), ('🚜', 400), ('😂', 386)]

5. VALORES EXTREMOS Y FECHAS
- Rango temporal: 2021-02-12 01:37:13+00:00 a 2021-02-24 09:22:35+00:00
- Tweets por fuera de 2021: 0


## Definición de estaratefias de optimización

Depues de analizar los datos y el código,ver detalle en la sección 3 del archivo [SOLUTION.md](../SOLUTION.md) 


## Calidad de software

Se proponen realizar test basados en los resultados del analisis de los datos. ver detalle en la sección 4 del archivo [SOLUTION.md](../SOLUTION.md.


## Código

El proceso de optimización de el procesamiento de datos empieza desde el proceso de lectura
del archivo.


In [ ]:
def read_standard_json(file_path: str) -> List[dict]:
    """Lee y devuelve una lista completa de diccionarios (Standard)."""
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return data


def read_streaming_orjson(file_path: str) -> Iterable[dict]:
    """Devuelve un generador (Lazy) de diccionarios usando orjson."""
    with open(file_path, "rb") as f:
        for line in f:
            try:
                yield orjson.loads(line)
            except orjson.JSONDecodeError:
                continue


def read_chunks_orjson(file_path: str, chunk_size: int = 5000) -> Iterable[List[dict]]:
    """Lee y entrega bloques de 5000 registros sin funciones extra."""
    with open(file_path, "rb") as f:
        chunk = []
        for line in f:
            try:
                chunk.append(orjson.loads(line))
                if len(chunk) == chunk_size:
                    yield chunk
                    chunk = []
            except orjson.JSONDecodeError:
                continue
        if chunk:
            yield chunk


def read_full_mem_json(file_path: str) -> List[dict]:
    """Lee todo el archivo en memoria y devuelve la lista de diccionarios."""
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return [json.loads(line) for line in lines if line.strip()]


def run_benchmarks(file_path: str):
    print(f"🚀 INICIANDO BENCHMARK INTEGRAL DESACOPLADO: {file_path}\n")

    strategies = [
        ("Standard JSON", read_standard_json),
        ("Streaming Orjson", read_streaming_orjson),
        ("Chunks Orjson", read_chunks_orjson),
        (
            "Polars read_ndjson (Eager)",
            lambda f: pl.read_ndjson(f, infer_schema_length=None, ignore_errors=True),
        ),
        (
            "Polars scan_ndjson (Lazy)",
            lambda f: pl.scan_ndjson(f, infer_schema_length=None, ignore_errors=True),
        ),
        (
            "Polars read_ndjson (Eager) + schema",
            lambda f: pl.read_ndjson(f, schema=twitter_schema, ignore_errors=True),
        ),
        (
            "Polars scan_ndjson (Lazy) + schema",
            lambda f: pl.scan_ndjson(f, schema=twitter_schema, ignore_errors=True),
        ),
        ("Full Memory (readlines)", read_full_mem_json),
        ("Pandas read_json", lambda f: pd.read_json(f, lines=True)),
    ]

    for name, func in strategies:
        print(f"\n{'#' * 70}")
        print(f"### ESTRATEGIA: {name}")
        print(f"{'#' * 70}")

        # Medición 1: Rendimiento Real (Tiempo Wall-clock + Pico RAM)
        # Esto es lo que realmente importa para la eficiencia base
        perf_monitor = profile_performance(func)
        perf_monitor(file_path)

        # Medición 2: Análisis Técnico (Opcional - Reporte cProfile)
        # Se ejecuta aparte para no influir en los tiempos reales de arriba
        detailed_monitor = profile_detailed(func)
        detailed_monitor(file_path)


run_benchmarks(file_path)

🚀 INICIANDO BENCHMARK INTEGRAL DESACOPLADO: ../farmers-protest-tweets-2021-2-4.json


######################################################################
### ESTRATEGIA: Standard JSON
######################################################################

[PERF] read_standard_json:
  > Tiempo: 9.8950 s
  > Memoria: 4500.62 MB

--- INICIANDO PERFILADO DETALLADO: read_standard_json ---
         1391837 function calls (1391820 primitive calls) in 10.344 seconds

   Ordered by: cumulative time
   List reduced from 154 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7    0.000    0.000   11.810    1.687 C:\Users\johanmarin\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\selectors.py:319(select)
        7    0.000    0.000   11.810    1.687 C:\Users\johanmarin\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\selectors.py:313(_select)
   117407    0.165    0.000    7.203    0.000 C:\Users\joha

Despues de haber definido los tipos de lectura de archivos que se utilizarán para cada proceso se procede a experimentar con el rendimiento de las funcions que ayudaran en la solución de las preguntas, para esto se definen funciones qeu permiten realizar las preubas más facilemnte

In [16]:
def text_chunk_reader(file_path: str, chunk_size: int = 5000) -> Iterable[List[bytes]]:
    with open(file_path, "rb") as f:
        chunk = []
        for line in f:
            chunk.append(line)
            if len(chunk) >= chunk_size:
                yield chunk
                chunk = []
        if chunk:
            yield chunk


@profile_performance
def lab_modular_test(
    name: str, reader_func: Callable, processor_func: Callable, path: str
):
    print(f"\n[LAB] Escenario: {name}")
    data = reader_func(path)
    return processor_func(data)


@profile_performance
def lab_parallel_test(name: str, file_path: str, mode: str = "process"):
    print(f"\n[LAB] Escenario: {name} ({mode.upper()})")
    total_counts = Counter()
    Executor = ProcessPoolExecutor if mode == "process" else ThreadPoolExecutor
    with Executor() as executor:
        results = executor.map(process_parallel_worker, text_chunk_reader(file_path))
        for local_counts in results:
            total_counts.update(local_counts)
    return len(total_counts)

### 

Dspues de definir los decoradores se preocede a realizar el benchmark para el codigo correspondiente a acad auna de las preguntas